# Movie Script

#### Short discription of what happens in this script: 

Accessing and use the "Get Details" and "Get Reviews" endpoints of "The Movie Database API" (https://developers.themoviedb.org/3/getting-started/introduction) to obtain the desired data:
* the movie ID (that was requested)
* the corresponding movie title
* the corresponding movie reviews

Writing this data into Kafka via the Kafka "movieProducer" & reading the requested movie ID via the Kafka "movieConsumer". (The movie ID is necessary to determine for which movie the reviews should be retrieved.)

#### The individual steps:

Install Kafka library:

In [1]:
#!pip install kafka-python

Import necessary packages:

In [2]:
import requests
from Hidden_Secret import myApiKey
import re
import json
from Kafka_Helpers import Producer, Consumer

Process data:

In [3]:
exampleID = 550
apiKey = myApiKey["apiKey"]

request1 = requests.get(f"https://api.themoviedb.org/3/movie/{exampleID}?api_key=105864a59e519ef281a74ca3af6c1b17&language=en-US")
response1 = request1.json()

request2 = requests.get(f"https://api.themoviedb.org/3/movie/{exampleID}/reviews?&language=en-US&page=1", {"api_key": apiKey})
response2 = request2.json()
result = response2['results']

# ID
movieID = [response1['id']]

# original_title
movieTitle = [response1['original_title']]

# content 
movieReviews = [item['content'] for item in result]
movieReviewsSplitted = [re.sub(r"[^\w \- \  ]", "", item.lower()).split(" ") for item in movieReviews] # ToDo: How to ignore empty string?

Open Producer & write data into Kafka:

In [4]:
movieProducer = Producer('localhost', 29092)

# movieProducer.send("topic_b", "key", json.dumps({
#     "movieID": exampleID, 
#     "movieReviewsSplitted": movieReviewsSplitted, 
#     "movieTitle": movieTitle
# }))

# ToDo: Adjust handler
def handler(key, value):
    detailsAndReviews = json.loads(value)
    movieID = detailsAndReviews["movieID"]
    movieTitle = detailsAndReviews["movieTitle"]
    movieReviewSplitted = detailsAndReviews["movieReviewsSplitted"]
    movieProducer.send("topic_b", key, {
    "movieID": exampleID, 
    "movieTitle": movieTitle,
    "movieReviewsSplitted": movieReviewsSplitted
})

Open Consumer & read data from Kafka:

In [5]:
movieConsumer = Consumer('localhost', 29092, "topic_a", handler)

Waiting for new events...
